# Distributed Computing
A distributed system is a system whose components are located on different networked computers, which communicate and coordinate their actions by passing messages to one another. Distributed computing is a field of computer science that studies distributed systems [link](https://en.wikipedia.org/wiki/Distributed_computing])

In [1]:
%%timeit
def square(x):
    return x * x

#The list command is to make a readable output
#in python3 map and filter create generators
list(map(square, range(10**7)))[:10]

1.25 s ± 63.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%%timeit
from functools import reduce

def square(x):
    return x * x

def add(x, y):
    return x + y

reduce(add, map(square, range(10**7)))

1.66 s ± 55.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
from functools import reduce

reduce(lambda x, y: x + y, map(lambda x: x * x, range(10**7)))

1.65 s ± 45.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Parallel Computing with Spark

In [ ]:
from pyspark import SparkContext

sc = SparkContext("local[*]", "temp")

In [7]:
lines = sc.textFile('./data/docs/')
lines.flatMap(lambda line: line.split(" ")) \
     .map(lambda word: (word.lower(), 1)) \
     .reduceByKey(lambda x, y: x + y) \
     .sortByKey() \
     .saveAsTextFile("counts")

In [8]:
! cat counts/part-00000 | grep "^('[a-z]" | head

('a', 13342)
('a!_', 1)
('a)', 9)
('a).', 1)
('a,', 10)
('a--e', 2)
('a--p.', 1)
('a--well,', 1)
('a-t-il', 1)
('a.', 93)
grep: write error: Broken pipe
cat: write error: Broken pipe


In [9]:
type(lines)

pyspark.rdd.RDD

In [12]:
print(lines.flatMap(lambda line: line.split(" ")),'\n',lines.flatMap(lambda line: line.split(" ")).count())

PythonRDD[17] at RDD at PythonRDD.scala:53 
 663351


### Spark DataFrame

In [20]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

df = lines.flatMap(lambda line: line.split(" ")) \
          .map(lambda word: (word.lower(), 1)) \
          .reduceByKey(lambda x, y: x + y) \
          .toDF(['word', 'count'])

In [22]:
df.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = true)



In [23]:
df.describe('count').show()

+-------+-----------------+
|summary|            count|
+-------+-----------------+
|  count|            74405|
|   mean|8.915408910691486|
| stddev|266.3647125527789|
|    min|                1|
|    max|            46691|
+-------+-----------------+



## Spark SQL

In [25]:
df.createOrReplaceTempView('counts')
sqlContext.sql('select * from counts where count > 5000').show()

+----+-----+
|word|count|
+----+-----+
| the|46691|
|  of|24986|
|    |34179|
|  it| 5819|
|  is| 7530|
| and|18239|
|that| 6452|
|  to|12389|
|   a|13342|
|  in|12717|
+----+-----+



### Machine Learning

In [27]:
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import UserDefinedFunction as udf
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.ml.linalg import VectorUDT, Vectors

In [28]:
word_length = udf(lambda x: Vectors.dense(len(x)), VectorUDT())
feat_df = df.withColumn("features", word_length("word")) \
            .withColumnRenamed("count", "label")

In [29]:
linreg = LinearRegression()
model = linreg.fit(feat_df)
model.coefficients

23/02/23 16:55:09 WARN Instrumentation: [54e32a0a] regParam is zero, which might cause numerical instability and overfitting.


DenseVector([-3.5131])